># QuantumSim - Shor's Nine Qubit Code
In this chapter an introduction is presented to quantum error correction (QEC) 

In [1]:
from quantumsim import *

## Bit-flip Correction Code
![Bit flip correction circuit](./pictures/BitFlipCorrectionCircuit.png "Bit flip correction circuit")

In [21]:
# Encode the logical qubit state among multiple qubits
bitFlipCorrectionCircuit = Circuit(3)
bitFlipCorrectionCircuit.pauli_x(0)
# Make init state: |100>

bitFlipCorrectionCircuit.cnot(0, 1)
bitFlipCorrectionCircuit.cnot(0, 2)
# |111>

# Create bitflip error
bitFlipCorrectionCircuit.bitflip_error_random()

# Decode, mayority check for correction
bitFlipCorrectionCircuit.cnot(0, 1)
bitFlipCorrectionCircuit.cnot(0, 2)
bitFlipCorrectionCircuit.toffoli(1, 2, 0)

bitFlipCorrectionCircuit.print_gates_and_descriptions()
bitFlipCorrectionCircuit.execute(print_state=False)
bitFlipCorrectionCircuit.measure(print_state=True)
# Measured answer must always be |1??>



X..	Pauli X on qubit 0
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
X..	Bit-flip error (Pauli X) on qubit 0
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
x**	Toffoli with control qubit 1 and CNOT with control qubit 2 and target qubit 0
Measured state:
|111>


'|111>'

## Phase-flip Correction Code
![Phase flip correction circuit](./pictures/PhaseFlipCorrectionCircuit.png "Phase flip correction circuit")

In [60]:
# Encode the logical qubit state among multiple qubits
phaseFlipCorrectionCircuit = Circuit(3)
phaseFlipCorrectionCircuit.pauli_x(0)
# Make init state: |100>

phaseFlipCorrectionCircuit.cnot(0, 1)
phaseFlipCorrectionCircuit.cnot(0, 2)
# |111>

phaseFlipCorrectionCircuit.hadamard(0)
phaseFlipCorrectionCircuit.hadamard(1)
phaseFlipCorrectionCircuit.hadamard(2)

# Create Phase-flip error
phaseFlipCorrectionCircuit.phaseflip_error_random()

phaseFlipCorrectionCircuit.hadamard(0)
phaseFlipCorrectionCircuit.hadamard(1)
phaseFlipCorrectionCircuit.hadamard(2)

# Decode, mayority check for correction
phaseFlipCorrectionCircuit.cnot(0, 1)
phaseFlipCorrectionCircuit.cnot(0, 2)
phaseFlipCorrectionCircuit.toffoli(1, 2, 0)

phaseFlipCorrectionCircuit.print_gates_and_descriptions()
phaseFlipCorrectionCircuit.execute(print_state=False)
phaseFlipCorrectionCircuit.measure(print_state=True)
# Measured answer must always be |1??>


X..	Pauli X on qubit 0
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
H..	Hadamard on qubit 0
.H.	Hadamard on qubit 1
..H	Hadamard on qubit 2
.Z.	Phase-flip error (Pauli Z) on qubit 1
H..	Hadamard on qubit 0
.H.	Hadamard on qubit 1
..H	Hadamard on qubit 2
*X.	CNOT with control qubit 0 and target qubit 1
*.X	CNOT with control qubit 0 and target qubit 2
x**	Toffoli with control qubit 1 and CNOT with control qubit 2 and target qubit 0
Measured state:
|110>


'|110>'

## Shor's Nine Qubit Code
![Shors nine qubit code](./pictures/ShorsNineQubitCodeCircuit.png "Shors nine qubit code")